In [1]:
import pandas as pd
import plotly.express as px

In [8]:
athlete_Event_Results = pd.read_csv('../Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['result_id','athlete','athlete_id','pos'])


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1

athlete_results.drop(repeat, inplace=True)
athlete_results.drop(columns=['edition_id', 'isTeamSport', 'index'], inplace=True)
athlete_results['edition'] = athlete_results['edition'].str.split().str[0].astype(int)

athlete_results["medal"] = athlete_results["medal"].notnull().mul(1)

gr = pd.DataFrame(athlete_results.groupby(['country_noc', 'sport', 'edition', 'event']).sum()).reset_index()

display(gr.head(20))

,country_noc,sport,edition,event,medal
0,AFG,Athletics,1936,"100 metres, Men",0
1,AFG,Athletics,1936,"Long Jump, Men",0
2,AFG,Athletics,1936,"Shot Put, Men",0
3,AFG,Athletics,1960,"1,500 metres, Men",0
4,AFG,Athletics,1960,"100 metres, Men",0
5,AFG,Athletics,1960,"110 metres Hurdles, Men",0
6,AFG,Athletics,1960,"200 metres, Men",0
7,AFG,Athletics,1960,"4 × 100 metres Relay, Men",0
8,AFG,Athletics,1960,"400 metres Hurdles, Men",0
9,AFG,Athletics,1960,"400 metres, Men",0


In [9]:
country = 'POR'
dataset = gr[gr['country_noc'] == country].reset_index()

dataset2 = dataset.copy()


new_medal = []
for i in range(len(dataset)):
    edition = dataset.loc[i, 'edition']
    sport = dataset.loc[i, 'sport']
    event = dataset.loc[i, 'event']

    new_medal.append(dataset[(dataset['edition'] <= edition) & (dataset['sport'] == sport) & (dataset['event'] == event)]['medal'].sum())

sports = pd.unique(dataset.sport)

dataset['medal'] = new_medal


print(dataset2[dataset2['sport'] == 'Athletics'])

     index country_noc      sport  edition               event  medal
101  79821         POR  Athletics     1912     100 metres, Men      0
102  79822         POR  Athletics     1912     200 metres, Men      0
103  79823         POR  Athletics     1912     400 metres, Men      0
104  79824         POR  Athletics     1912     800 metres, Men      0
105  79825         POR  Athletics     1912       Marathon, Men      0
..     ...         ...        ...      ...                 ...    ...
347  80067         POR  Athletics     2020     Marathon, Women      0
348  80068         POR  Athletics     2020       Shot Put, Men      0
349  80069         POR  Athletics     2020     Shot Put, Women      0
350  80070         POR  Athletics     2020    Triple Jump, Men      1
351  80071         POR  Athletics     2020  Triple Jump, Women      1

[251 rows x 6 columns]


In [10]:
dataset2 = pd.DataFrame(dataset2.groupby(['sport']).sum()).reset_index()

dataset2 = dataset2.sort_values(by=['medal'], ascending=False)

dataset2 = dataset2.head(5)

color = ['#0081C8', '#FCB131', '#000000', '#00A651', '#EE334E']
dataset2['color'] = color

print(dataset2)

fig = px.bar(dataset2, x="sport", y="medal", color= 'color', title="Top 5 sports for the country", width=300, height=250,
             color_discrete_sequence=['#0081C8', '#FCB131', '#000000', '#00A651', '#EE334E'] )
fig.update_layout(font_family= 'Cabin',autosize = False, 
                      legend=dict(yanchor="top", xanchor="left", font=dict(size=15)),
                      margin=dict(l=1, r=6, b=20, t=31, pad=0), showlegend=False,)
fig.show()

                 sport     index  edition  medal    color
3            Athletics  20066446   498992     12  #0081C8
26             Sailing   6027300   148908      4  #FCB131
20                Judo   4976523   123964      3  #000000
8         Canoe Sprint   2883942    72188      2  #00A651
15  Equestrian Jumping   1844186    45140      2  #EE334E


In [4]:
from dash import dcc, html, Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)

colors = {
    'background': '#EEF1FA'
}
text = {
    'font_family': 'Cabin'
}

app.layout = html.Div(style={'backgroundColor': '#EEF1FA', 'font-family': 'Cabin'}, children = [
    html.H4('Sports evolution of ' + country),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=sports,
        value=[sports[0]],
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))

def update_line_chart(s):
    df = dataset#px.data.gapminder()
    mask = df.sport.isin(s)
    fig = px.line(df[mask], 
        x="edition", y="medal", color='event').update_layout({
                                                    'plot_bgcolor': '#FFFFFF',
                                                    'paper_bgcolor': '#EEF1FA'
                                                    }, font_family= 'Cabin'
                                                    )
    fig.update_xaxes(showline=True, linewidth=2, gridcolor='#c7c7c7')
    fig.update_yaxes(showline=True, linewidth=2, gridcolor='#c7c7c7')
    return fig


app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
